### Task 1 - Data Scraping and Collection (Extract & Load)

In [15]:
# %pip install telethon
# %pip install psycopg2
# %pip install pandas
# %pip install matplotlib
# %pip install numpy
# %pip install psycopg2
# %pip install dotenv

In [1]:
import os
import sys
import asyncio
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from telethon.sync import TelegramClient
from dotenv import load_dotenv
from telethon import TelegramClient
from telethon.tl.types import MessageMediaPhoto
import numpy as np
import json
from pathlib import Path
from datetime import datetime
# Load .env file from the project root
load_dotenv(dotenv_path="../.env")  # adjust the path if needed
import logging
sys.path.append(os.path.abspath("../Script"))
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.chdir("..") 
print(os.getcwd())

d:\KAIM 5,6,7\Week-7\Technical\Shipping-Data-from-Raw-Telegram-Data


In [3]:
# Access the variables
api_id = os.getenv("tg_api_id")
api_hash = os.getenv("tg_api_hash")
session_name = os.getenv("session_name")

channels = {
    "lobelia4cosmetics": "https://t.me/lobelia4cosmetics",
    "tikvahpharma": "https://t.me/tikvahpharma"
    # Add more from et.tgstat.com/medicine
}

raw_dir = Path("Data/raw/telegram_messages")

logging.basicConfig(filename="scrape.log", level=logging.INFO)

async def scrape_channel(channel_name, max_msgs=1000):
    client = TelegramClient("async_session", api_id, api_hash)
    try:
        await client.start()
        messages = []
        async for msg in client.iter_messages(channel_name, limit=max_msgs):
            messages.append(msg.to_dict())
        logging.info(f"[{channel_name}] Scraped {len(messages)} messages.")

        # Save JSON
        today = datetime.now().strftime("%Y-%m-%d")
        out_dir = raw_dir / today / channel_name
        out_dir.mkdir(parents=True, exist_ok=True)
        with open(out_dir / "messages.json", "w", encoding="utf-8") as f:
            json.dump(messages, f, ensure_ascii=False, indent=2)

    except Exception as e:
        logging.error(f"Error scraping {channel_name}: {e}")

In [23]:
await run()

In [7]:
from data_scraper import get_pgsql_connection

In [8]:
# Async image scraping function
async def scrape_images_async(channel_username, limit=50):
    client = TelegramClient(session_name, api_id, api_hash)
    await client.start()
    print(f"Connected to Telegram. Scraping images from {channel_username}...")

    conn = get_pgsql_connection()
    if conn is None:
        return
    cur = conn.cursor()

    save_dir = os.path.join("images", channel_username.strip("@"))
    os.makedirs(save_dir, exist_ok=True)

    try:
        async for message in client.iter_messages(channel_username,
limit=limit):
            if message.media and isinstance(message.media, MessageMediaPhoto):
                try:
                    file_name = f"{message.id}.jpg"
                    file_path = os.path.join(save_dir, file_name)
                    await client.download_media(message, file_path)

                    cur.execute("""
                        INSERT INTO images (message_id, channel_name,file_name, file_path, posted_at)
                        VALUES (%s, %s, %s, %s, %s)
                        ON CONFLICT DO NOTHING
                    """, (
                        message.id,
                        channel_username,
                        file_name,
                        file_path,
                        message.date
                    ))

                    print(f"Saved image: {file_name}")
                except Exception as insert_err:
                    print(f"Error inserting {message.id}: {insert_err}")

        conn.commit()
    except Exception as scrape_err:
        print(f"Scraping failed: {scrape_err}")
    finally:
        cur.close()
        conn.close()
        await client.disconnect()
        print("Disconnected from Telegram.")

In [9]:
await scrape_images_async("@lobelia4cosmetics", limit=30)

OperationalError: database is locked